## Semidefinite program to identify the POVMS for Minimum error discrimination and Cloning for the DPS QKD states

This notebook contains the code to find the POVMS corresponding to the minimum error descrimination of the DPS QKD states. The results of which are used in the paper "Explicit attacks on differential phase shifted quantum key distribution"


This notebook depends upon various packages including numpy >= 1.19.5, picos >= 2.2.55, and cvxopt >= 1.2.5.

In [47]:
# Necessary inputs 
import cvxopt as cvx
import picos as pic
import numpy as np
import math
import matplotlib.pyplot as plt


In [48]:
print('Solvers supported on this installation of PICOS:', pic.solvers.all_solvers().keys())

Solvers supported on this installation of PICOS: dict_keys(['cplex', 'cvxopt', 'ecos', 'glpk', 'gurobi', 'mosek', 'mskfsn', 'osqp', 'scip', 'smcp'])


In [49]:
print('Solvers available to PICOS on this machine :', pic.solvers.available_solvers())

Solvers available to PICOS on this machine : ['cvxopt', 'ecos', 'mosek', 'mskfsn', 'osqp']


## MED SDP
### The 3-pulse DPS states that we are trying to descriminate can be written in the vector form as, 
$$\begin{array}{ll}
\psi (+,+) = (1/\sqrt{3})[0;1;1;0;1;0;0;0];\\ 
\psi (+,-)= (1/\sqrt{3})[0;-1;1;0;1;0;0;0];\\ 
\psi (-,+) = (1/\sqrt{3})[0;1;-1;0;1;0;0;0];\\  
\psi (-,-) = (1/\sqrt{3})[0;-1;-1;0;1;0;0;0]. 
\end{array}$$
### The SDP corresponding to the DPS states minimum error discrimination is-
\begin{equation}
    \begin{array}{ll}
\text{ maximize:} & \frac{1}{4}\langle \rho_i, P_i\rangle \\
  \text {subject to:}  &\sum_iP_i = I \\ 
& P_i>0
\end{array} \label{eq:sdp_main}
\end{equation}
Here, $\rho_i$ is the density matrix corresponding to the DPS states.

In [53]:
# kets is an array that holds all the DPS states sent by Alice
kets=[(1/np.sqrt(3))*np.matrix([0,1,1,0,1,0,0,0]),(1/np.sqrt(3))*np.matrix([0,-1,1,0,1,0,0,0]),(1/np.sqrt(3))*np.matrix([0,1,-1,0,1,0,0,0]), (1/np.sqrt(3))*np.matrix([0,-1,-1,0,1,0,0,0])]


In [68]:
# rhos contian the density matrices corresponding to the DPS QKD states
rhos=[]
for i in range(len(kets)):
    rhos.append(np.matrix(np.transpose(kets[i])*kets[i]))




In [70]:
#The constants in the SDP 
Sgs1 = pic.Constant("sg1", rhos[0])
Sgs2 = pic.Constant("sg2", rhos[1])
Sgs3 = pic.Constant("sg3", rhos[2])
Sgs4 = pic.Constant("sg4", rhos[3])

# q here is the probability with which each state is sent 
q = pic.Constant("q", 0.25)

#Identity matrix
shp = np.shape(rhos[1])
iMat = pic.Constant('I', np.eye(8,dtype='complex'))

#Variables - here are the POVMS 
#----------
eVars1 = pic.HermitianVariable("E1", shp)
eVars2 = pic.HermitianVariable("E2", shp)
eVars3 = pic.HermitianVariable("E3", shp)





prob1 = pic.Problem()
    
#Constraints - The POVMS are positiove operators and they sum to identity of appropriate dimension
#----------
prob1.add_constraint(eVars1 >> 0)
prob1.add_constraint(eVars2 >> 0)
prob1.add_constraint(eVars3 >> 0)
prob1.add_constraint(iMat -eVars1 - eVars2 - eVars3  >> 0)

#Objective
#----------
obj = q*(Sgs1 | eVars1) + q*(Sgs2 | eVars2) + q*(Sgs3 | eVars3) + (1-3*q)*(Sgs4 |(iMat -eVars1 - eVars2 - eVars3) )

prob1.set_objective('max',obj)

In [72]:
# Printing the SDP formulated.
print(prob1)

Complex Semidefinite Program
  maximize q·⟨sg1, E1⟩ + q·⟨sg2, E2⟩ + q·⟨sg3, E3⟩ + (1 - 3·q)·⟨sg4, I - E1 - E2 - E3⟩
  over
    8×8 hermitian variable Ei+1 ∀ i ∈ [0…2]
  subject to
    E1 ≽ 0
    E2 ≽ 0
    E3 ≽ 0
    I - E1 - E2 - E3 ≽ 0


In [74]:
# Solving the problem using cvxopt
prob1.solve(verbosity=False,solver='cvxopt')

<primal feasible solution pair (claimed optimal) from cvxopt>

In [75]:
# printing the optimized value
prb = prob1.value
print('Probability of discriminating the inputs is = ', prb)

Probability of discriminating the inputs is =  0.7499999998232117


## Cloning SDP

Here we optimize and find the Chi matrix for which the fidelity with the state sent by ALice is high. The theory can be found in the paper.

In [81]:
# the 3-pulse DPS state as qutrit states 
ketsclon=[(1/np.sqrt(3))*np.matrix([1,1,1]),(1/np.sqrt(3))*np.matrix([1,-1,1]), (1/np.sqrt(3))*np.matrix([1,1,-1]),(1/np.sqrt(3))*np.matrix([1,-1,-1])]

In [82]:
#According to the SDP, we need the \psi \otimes \psi \otimes \psi.
tensorketclon=[]
for i in range(len(ketsclon)):
    tensorketclon.append(np.kron(ketsclon[i],np.kron(ketsclon[i],ketsclon[i])))

In [86]:
#constant in the SDP
c=0
for i in  range(len(tensorketclon)):
    c=c + (0.25*np.matrix(np.transpose(tensorketclon[i])*tensorketclon[i]))

In [100]:
# Formulating the SDP
C=pic.Constant("C",c)
X=pic.HermitianVariable("X",(27,27))
iMat3 = pic.Constant('I', np.eye(3,dtype='complex'))
prob2=pic.Problem()
prob2.add_constraint(X>>0)
prob2.add_constraint((X[0:3,0:3] + X[3:6,3:6] + X[6:9,6:9] + X[9:12,9:12]+ X[12:15,12:15]+ X[15:18,15:18]+ X[18:21,18:21]+ X[21:24,21:24]+ X[24:27,24:27])==iMat3)
obj2=(C|X)
prob2.set_objective('max',obj2)

In [101]:
print(prob2)

Complex Semidefinite Program
  maximize ⟨C, X⟩
  over
    27×27 hermitian variable X
  subject to
    X ≽ 0
    X[0:3,0:3] + X[3:6,3:6] + X[6:9,6:9] + X[9:12,9:12] +
      X[12:15,12:15] + X[15:18,15:18] + X[18:21,18:21] +
      X[21:24,21:24] + X[24:27,24:27] = I


In [102]:
prob2.solve(verbosity=False,solver='cvxopt')

<primal feasible solution pair (claimed optimal) from cvxopt>

In [104]:
# printing the optimized value
prb2 = prob2.value
print('Probability of right cloning of the inputs is = ', prb2)

Probability of right cloning of the inputs is =  0.7777777775046744
